In [1]:
import pandas as pd
import plotly.express as px

In [2]:
# Read data from xlsx file
df = pd.read_excel('Anytime Fitness.xlsx', sheet_name='Sheet1')

df[['Location', 'Region']] = df['Location'].str.split(', ', expand=True)

# Remove all "GMT+8" from the "Time" column and strip the spaces
df['Time'] = df['Time'].str.replace('GMT+8', '').str.strip()

# Merge the "Date" and "Time" columns into one column
df['Datetime'] = df['Date'].astype(str) + ' ' + df['Time'].astype(str)

# Drop the "Date" and "Time" columns
df.drop(['Date', 'Time'], axis=1, inplace=True)

# Convert the 'date_column' to datetime
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%B %d, %Y %H:%M')

# Create a "Month" column by Month name, e.g. Jan, Feb, Mar, etc.
df['Month'] = df['Datetime'].dt.strftime('%b')

# Create a "Hour" column by 12-hour format
df['Hour'] = df['Datetime'].dt.hour

df

,Location,Region,Datetime,Month,Hour
0,Elias CC,North East,2023-12-28 16:55:00,Dec,16
1,Elias CC,North East,2023-12-26 22:34:00,Dec,22
2,Elias CC,North East,2023-12-25 18:03:00,Dec,18
3,Elias CC,North East,2023-12-23 12:49:00,Dec,12
4,Elias CC,North East,2023-12-17 18:38:00,Dec,18
...,...,...,...,...,...
111,Elias CC,North East,2023-01-20 06:35:00,Jan,6
112,Elias CC,North East,2023-01-19 12:17:00,Jan,12
113,Elias CC,North East,2023-01-17 06:09:00,Jan,6
114,Elias CC,North East,2023-01-16 06:28:00,Jan,6


In [3]:
# Count the number of check-ins
total_visits = df["Location"].count()
print(f"Total Visits: {total_visits}")


Total Visits: 116


In [6]:
# Plot the number of check-ins by month
fig = px.histogram(df, x="Month", title="Number of Check-ins by Month")

fig.update_xaxes(categoryorder='array', categoryarray= ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Change plotly theme
fig.layout.template = 'plotly_white'

# Change y-axis title
fig.update_yaxes(title_text='Visit')

# Remove x-axis title
fig.update_xaxes(title_text=None)

# Change title
fig.update_layout(title_text='Number of Visits by Month')

# Change the bar color
fig.update_traces(marker_color='#6046b1')

# Change the font to conthrax
fig.update_layout(font_family='Conthrax')

# Change hoverlabel information
fig.update_traces(hovertemplate='Month: %{x}<br>Visits: %{y}')

# Change figute size
fig.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=100,
        pad=4
    ),
)

fig.show()

# Save the image
fig.write_image("Number of Visits by Month.png")

In [5]:
# Create a DataFrame with all 24 hours
all_hours = pd.DataFrame({'Hour': range(24)})

# Merge the DataFrames to include all hours
merged_df = all_hours.merge(df.groupby('Hour')['Datetime'].count(), on='Hour', how='left')

# Rename the column for clarity
merged_df.rename(columns={'Datetime': 'Visit'}, inplace=True)

# Fill the NaN values with 0
merged_df.fillna(0, inplace=True)

# Plot the number of check-ins by hour
fig = px.bar(merged_df, x='Hour', y='Visit')

# Show all labels on the x-axis
fig.update_xaxes(tickmode='linear')

# Change plotly theme
fig.layout.template = 'plotly_white'

# Change y-axis title
fig.update_yaxes(title_text='Visit')

# Remove x-axis title
fig.update_xaxes(title_text='24-Hour')

# Change title
fig.update_layout(title_text='Number of Visits by Hour')

# Change the bar color
fig.update_traces(marker_color='#6046b1')

# Change the font to conthrax
fig.update_layout(font_family='Conthrax')

# Change hoverlabel information
fig.update_traces(hovertemplate='Hour: %{x}<br>Visits: %{y}')

# Change figute size
fig.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=100,
        pad=4
    ),
)


fig.show()

# Save the image
fig.write_image("Number of Visits by Hour.png")

In [84]:
import folium

## CREATE MAP OF SINGAPORE
# Coordinates for the central location in Singapore
latitude = 1.33
longitude = 103.8198

# Build the default map for Singapore
map_singapore = folium.Map(location=[latitude, longitude], tiles="cartodb positron", zoom_start=11.45)


## ADD MARKERS TO THE MAP
coordinates_dict = {
   'Elias CC': '1.3786522389514384, 103.94260712614648',
   'Balestier': '1.3237206779610273, 103.85280419976769',
   'Bugis': '1.2975689769071006, 103.8552854414898',
   'Chai Chee': '1.3234588181493, 103.92092999731074',
   'Pasir Ris E!Hub': '1.3760431421725428, 103.95542719731068',
   'Changi City Point': '1.334714472716219, 103.96305239731065',
   'Loyang Point': '1.3672751462936503, 103.9646091991615',
   'Punggol Oasis': '1.4033299109521162, 103.91311360895263',
   'Eastpoint Mall': '1.343051416422387, 103.95314378381829',
   'Tanjong Pagar': '1.2763195007264114, 103.84585595313155',
   'Grantral Complex': '1.3341565949945466, 103.88817709546002',
   'Orchard': '1.3021333938779398, 103.83903902660971'
}

# Create a DataFrame from the dictionary with two columns "Location" and "Coordinates"
coordinates_df = pd.DataFrame(list(coordinates_dict.items()), columns=['Location', 'Coordinates'])

# Split the "Coordinates" column into two columns
coordinates_df[['Lat', 'Lon']] = coordinates_df['Coordinates'].str.split(',', expand=True)

# Drop the "Coordinates" column
coordinates_df.drop('Coordinates', axis=1, inplace=True)

# Convert the "Latitude" and "Longitude" columns to float
coordinates_df['Lat'] = coordinates_df['Lat'].astype(float)
coordinates_df['Lon'] = coordinates_df['Lon'].astype(float)

# Group by "Location" and count the number of check-ins
location_df = df.groupby('Location')['Datetime'].count().reset_index()

# Rename the column for clarity
location_df.rename(columns={'Datetime': 'Visit'}, inplace=True)

# Merge the DataFrames to include the coordinates
location_df = location_df.merge(coordinates_df, on='Location', how='left')

location_df['Adjusted_Radius'] = location_df['Visit'].apply(lambda x: math.log(x + 1))

import math
# add marker one by one on the map, and account for Mercator deformation
for location in location_df.itertuples():
    local_deformation = math.cos(location.Lat * math.pi / 180)
    folium.Circle(
        location=[location.Lat, location.Lon],
        popup='%s: %.0f' % (location.Location, location.Visit),
        radius=location.Adjusted_Radius * 600.0 * local_deformation,
        color='#6046b1',
        fill=True,
        fill_color='#6046b1'
    ).add_to(map_singapore)

# Show the map again
map_singapore

# Export the map as HTML
map_singapore.save('map.html')


In [8]:
total_cost_year = 95 * 12
cost_per_visit = total_cost_year / total_visits

print(f"Cost per Visit: {cost_per_visit:.2f}")

Cost per Visit: 9.83
